Setup

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
#from sklearn.cross_validation import train_test_split 
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy.polynomial.polynomial as poly
%matplotlib inline

df = pd.read_csv("qudditch_training.csv",)

In [55]:
#data = (df[~np.isnan(df).any(axis=1)])

print(df.shape)
data = df.values
X0 = data[:,2:47]
X1 = np.delete(X0,[3,8,18,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43],1)
#X1:house, gender, age, foul_type_id, game_move_id,penalty_id,game_duration,move_speciality,num_game_moves,
#   num_game_losses,num_practice_sessions,num_games_satout,num_games_injured,num_games_notpartof,player_type,
#   num_games_won,stooging?,body_blow?,power_play?,sloth_grip_roll?,change,snitch_caught

Y0 = data[:,-1]
print(X1.shape)

#Y0 = np.array(Y0)
Y = Y0=='YES'
Y = Y.reshape((Y.shape[0],1))
print(Y.shape)

gryffindor = np.zeros((X1.shape[0],1))
hufflepuff = np.zeros((X1.shape[0],1))
ravenclaw = np.zeros((X1.shape[0],1))
slytherin = np.zeros((X1.shape[0],1))

female = np.zeros((X1.shape[0],1))
male = np.zeros((X1.shape[0],1))

for row in range(len(X1)):
    if X1[row][0]=="Gryffindor":
        gryffindor[row]=True
    elif X1[row][0]=="Hufflepuff":
        hufflepuff[row]=True
    elif X1[row][0]=="Ravenclaw":
        ravenclaw[row]=True
    elif X1[row][0]=="Slytherin":
        slytherin[row]=True
    if X1[row][1]=="Female":
        female[row]=True
    elif X1[row][1]=="Male":
        male[row]=True
        
X1 = np.hstack((X1,gryffindor,hufflepuff,ravenclaw,slytherin,female,male))
X = np.delete(X1,[0,1],1)
print(X[0])

print(X[0:20,-6:-2])
print(X[0:20,-2:])

(101268, 48)
(101268, 20)
(101268, 1)
Female
[ 1.]
[[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]]
[11.0 6.0 25.0 1.0 1.0 '38' 41.0 0.0 1.0 0.0 0.0 0.0 'Beater' 1.0 'None'
 'No' 'No' 'No' 1.0 0.0 0.0 0.0 1.0 0.0]
[[1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.0]
 [0.0 0.0 0.0 0.0]]
[[1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [1.0 0.0]
 [1.0 0.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]]


Logistic Regression

Support Vector Machines

Neural Networks